In [4]:
import json
import torch
from sklearn.model_selection import train_test_split
from transformers import XLMRobertaTokenizer, XLMRobertaForSequenceClassification, Trainer, TrainingArguments
from sklearn.metrics import precision_recall_fscore_support, accuracy_score
import numpy as np

# Lade den neuen Datensatz mit Rumor-Evidence-Label-Paaren
with open('/content/rumor_evidence_label_pairs.json', 'r') as f:
    data = json.load(f)

# Extrahiere Rumor, Evidence und Labels aus dem Datensatz
rumor_texts = [item['rumor'] for item in data]
evidence_texts = [item['evidence'] for item in data]
labels = [item['label'] for item in data]

# Kombiniere Rumor und Evidence klar definiert für das Modell-Training
combined_texts = [f"Rumor: {rumor} [SEP] Evidence: {evidence}" for rumor, evidence in zip(rumor_texts, evidence_texts)]

# Konvertiere Label-Texte in numerische Werte
label_to_id = {'SUPPORTS': 0, 'REFUTES': 1, 'NOT ENOUGH INFO': 2}
numerical_labels = [label_to_id[label] for label in labels]

# Splitte den Datensatz in Trainings- und Testdaten
train_texts, test_texts, train_labels, test_labels = train_test_split(combined_texts, numerical_labels, test_size=0.2)

# Lade den Tokenizer
tokenizer = XLMRobertaTokenizer.from_pretrained('xlm-roberta-base')

# Tokenisiere die Daten
def tokenize_function(texts):
    return tokenizer(texts, padding="max_length", truncation=True, max_length=256)

train_encodings = tokenize_function(train_texts)
test_encodings = tokenize_function(test_texts)

# Konvertiere Labels in PyTorch-Tensoren
train_labels = torch.tensor(train_labels)
test_labels = torch.tensor(test_labels)

# Erstelle PyTorch-Datensätze
class RumorStanceDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = self.labels[idx]
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = RumorStanceDataset(train_encodings, train_labels)
test_dataset = RumorStanceDataset(test_encodings, test_labels)

# Lade das XLM-RoBERTa-Modell für die Klassifikation
model = XLMRobertaForSequenceClassification.from_pretrained('xlm-roberta-base', num_labels=3)

# Definiere die Trainingsparameter
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=10,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    logging_dir='./logs',
    evaluation_strategy="epoch",
)

# Berechne die Metriken (Precision, Recall, F1, Accuracy)
def compute_metrics(pred):
    labels = pred.label_ids
    preds = np.argmax(pred.predictions, axis=1)

    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='weighted', zero_division=0)
    acc = accuracy_score(labels, preds)

    return {
        'accuracy': acc,
        'precision': precision,
        'recall': recall,
        'f1': f1
    }

# Erstelle den Trainer mit den Metriken
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

# Starte das Training
trainer.train()

# Speichere das feinabgestimmte Modell
model.save_pretrained('/content/fine-tuned-xlm-roberta')
tokenizer.save_pretrained('/content/fine-tuned-xlm-roberta')

# Führe die Bewertung des Modells durch
results = trainer.evaluate()
print(f"Results: {results}")
print(f"Accuracy: {results['eval_accuracy']}")
print(f"Precision: {results['eval_precision']}")
print(f"Recall: {results['eval_recall']}")
print(f"F1-Score: {results['eval_f1']}")


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,No log,1.017071,0.456790,0.208657,0.456790,0.286462
2,No log,0.560156,0.691358,0.811728,0.691358,0.691588
3,No log,0.232947,0.938272,0.939983,0.938272,0.937782
4,No log,0.304086,0.938272,0.939492,0.938272,0.938596
5,No log,0.204872,0.962963,0.964506,0.962963,0.963018
6,No log,0.194528,0.975309,0.976530,0.975309,0.975280
7,No log,0.199721,0.975309,0.976530,0.975309,0.975280
8,No log,0.199855,0.975309,0.976530,0.975309,0.975280
9,No log,0.201592,0.975309,0.976530,0.975309,0.975280
10,No log,0.201618,0.975309,0.976530,0.975309,0.975280


Results: {'eval_loss': 0.20161807537078857, 'eval_accuracy': 0.9753086419753086, 'eval_precision': 0.976529934952874, 'eval_recall': 0.9753086419753086, 'eval_f1': 0.9752797570721486, 'eval_runtime': 1.2388, 'eval_samples_per_second': 65.384, 'eval_steps_per_second': 8.879, 'epoch': 10.0}
Accuracy: 0.9753086419753086
Precision: 0.976529934952874
Recall: 0.9753086419753086
F1-Score: 0.9752797570721486


In [5]:
!zip -r /content/fine-tuned-xlm-roberta.zip /content/fine-tuned-xlm-roberta


updating: content/fine-tuned-xlm-roberta/ (stored 0%)
updating: content/fine-tuned-xlm-roberta/sentencepiece.bpe.model (deflated 49%)
updating: content/fine-tuned-xlm-roberta/model.safetensors (deflated 31%)
updating: content/fine-tuned-xlm-roberta/config.json (deflated 52%)
updating: content/fine-tuned-xlm-roberta/tokenizer_config.json (deflated 76%)
updating: content/fine-tuned-xlm-roberta/special_tokens_map.json (deflated 52%)


In [6]:
from google.colab import files
files.download('/content/fine-tuned-xlm-roberta.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>